In [ ]:
!pip install kafka-python

In [2]:
import json
import requests
from requests.exceptions import RequestException
import time 
from kafka import KafkaProducer

In [3]:
bootstrap_server = 'broker:29092'
api = 'Add Your API Key Here'

In [4]:
producer = KafkaProducer(bootstrap_servers=bootstrap_server,
                        value_serializer = lambda v : json.dumps(v).encode('utf-8'))

In [5]:
def weather_data_request (city):
    url = f'https://api.openweathermap.org/data/2.5/weather?q={city}&appid={api}'
    for i in range(3):
        try:
            response = requests.get(url)
            response.raise_for_status()  # Raise an exception for HTTP errors
            raw_data = response.json()
            weather_data = {
                'city_id': raw_data['id'],
                'city': raw_data['name'],
                'weather_condition': raw_data['weather'][0]['main'],
                'weather_description': raw_data['weather'][0]['description'],
                'temperature': raw_data['main']['temp'],
                'min_temp': raw_data['main']['temp_min'],
                'max_temp': raw_data['main']['temp_max'],
                'pressure': raw_data['main']['pressure'],
                'humidity': raw_data['main']['humidity'],
                'wind_speed': raw_data['wind']['speed'],
                'visibility': raw_data['visibility']
            }
            return weather_data
        except RequestException as e:
            print(f"Error retrieving weather data for {city}: {e}")
            print("Retrying in 5 seconds...")
            time.sleep(5)
    return None

In [6]:
def correct_time_zone (timestamp):
    import datetime
    import pytz
    time_zone  = pytz.timezone('EET')
    local_time = datetime.datetime.fromtimestamp(timestamp, tz=time_zone).strftime('%Y-%m-%d %H:%M:%S')
    return local_time

In [7]:
def add_creation_time (weather_data):
    timestamp = time.time()
    weather_data['creation_time']=correct_time_zone(timestamp)
    return weather_data

In [8]:
def get_weather_data(cities):
    # cities = ['London', 'Paris']
    cities_weather_data = {}
    for city in cities:
        cities_weather_data[city] = weather_data_request(city)
        time.sleep(1)
    for city in cities:
        add_creation_time(cities_weather_data[city])
    return cities_weather_data


In [ ]:
while True:
    cities = ["Gaza", "Rafah", "Cairo", "Mecca", "Aleppo"]
    final_data = get_weather_data(cities)
    producer.send('weather_data',final_data["Gaza"])
    producer.send('weather_data',final_data["Rafah"])
    producer.send('weather_data',final_data["Cairo"])
    producer.send('weather_data',final_data["Mecca"])
    producer.send('weather_data',final_data["Aleppo"])
    time.sleep(10)